In [86]:
import pandas as pd

In [88]:
df=pd.read_csv("C:\\Users\\aseks\\Downloads\\Software Engineer Salaries - Software Engineer Salaries.csv")

In [90]:
df.head(5)

,Company,Company Score,Job Title,Location,Date,Salary
0,ViewSoft,4.8,Software Engineer,"Manassas, VA",8d,$68K - $94K (Glassdoor est.)
1,Workiva,4.3,Software Support Engineer,Remote,2d,$61K - $104K (Employer est.)
2,"Garmin International, Inc.",3.9,C# Software Engineer,"Cary, NC",2d,$95K - $118K (Glassdoor est.)
3,Snapchat,3.5,"Software Engineer, Fullstack, 1+ Years of Expe...","Los Angeles, CA",2d,$97K - $145K (Employer est.)
4,Vitesco Technologies Group AG,3.1,Software Engineer,"Seguin, TX",2d,$85K - $108K (Glassdoor est.)


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        866 non-null    object 
 1   Company Score  788 non-null    float64
 2   Job Title      865 non-null    object 
 3   Location       852 non-null    object 
 4   Date           865 non-null    object 
 5   Salary         759 non-null    object 
dtypes: float64(1), object(5)
memory usage: 40.9+ KB


In [94]:
df.describe()

,Company Score
count,788.000000
mean,3.895051
std,0.525235
min,1.000000
25%,3.600000
50%,3.900000
75%,4.200000
max,5.000000


In [96]:
missing_values = df.isnull().sum()
print(missing_values)

Company            4
Company Score     82
Job Title          5
Location          18
Date               5
Salary           111
dtype: int64


In [126]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df_cleaned = df.copy()
df_cleaned[numeric_columns] = df_cleaned[numeric_columns].fillna(df_cleaned[numeric_columns].mean())


In [128]:
non_numeric_columns = df.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns:
    df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mode()[0])

In [130]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [134]:
def remove_outliers_iqr(df):
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    Q1 = numeric_df.quantile(0.25)
    Q3 = numeric_df.quantile(0.75)
    IQR = Q3 - Q1
    df_outliers_removed = df[~((numeric_df < (Q1 - 1.5 * IQR)) | (numeric_df > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_outliers_removed

df_no_outliers = remove_outliers_iqr(df_cleaned)

In [136]:
scaler = MinMaxScaler()
df_no_outliers.loc[:, numeric_columns] = scaler.fit_transform(df_no_outliers.loc[:, numeric_columns])

In [140]:
label_encoder = LabelEncoder()
df_no_outliers.loc[:, 'Company'] = label_encoder.fit_transform(df_no_outliers.loc[:, 'Company'])
df_no_outliers.loc[:, 'Job Title'] = label_encoder.fit_transform(df_no_outliers.loc[:, 'Job Title'])
df_no_outliers.loc[:, 'Location'] = label_encoder.fit_transform(df_no_outliers.loc[:, 'Location'])
df_no_outliers.loc[:, 'Salary'] = label_encoder.fit_transform(df_no_outliers.loc[:, 'Salary'])

In [142]:
X = df_no_outliers.drop(columns=['Salary'])
y = df_no_outliers['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(df_no_outliers.head())

  Company  Company Score Job Title Location Date Salary
0     571       1.000000       294      174   8d    358
1     598       0.736842       466      243   2d    320
2     230       0.526316        40       55   2d    598
3     481       0.315789       418      168   2d    618
4     575       0.105263       294      267   2d    500


In [144]:
df_no_outliers.to_csv('output_filename.csv', index=False)

In [178]:
df_no_outliers.loc[:, 'Date'] = df_no_outliers.loc[:, 'Date'].astype(str)

df_no_outliers.loc[:, 'Date'] = df_no_outliers.loc[:, 'Date'].str.extract(r'(\d+)')
df_no_outliers.loc[:, 'Date'] = pd.to_numeric(df_no_outliers.loc[:, 'Date'], errors='coerce')
df_no_outliers.loc[:, 'Date'] = df_no_outliers.loc[:, 'Date'].fillna(0).astype(float)

X = df_no_outliers.drop(columns=['Salary'])
y = df_no_outliers['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

ridge_model = Ridge()

# Hyperparameter Grid
param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sag', 'saga']
}

grid_search = GridSearchCV(estimator=ridge_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
y_pred = grid_search.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print("Best Cross-Validation Score:", best_score)
print("Test MSE:", test_mse)

C:\Users\aseks\AppData\Local\Temp\ipykernel_12680\396687761.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_no_outliers.loc[:, 'Date'] = df_no_outliers.loc[:, 'Date'].fillna(0).astype(float)


Best Hyperparameters: {'alpha': 100, 'solver': 'lsqr'}
Best Cross-Validation Score: -33539.76000228875
Test MSE: 33482.23268286485


In [180]:
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r2)


Mean Absolute Error (MAE): 162.63608205844002
R-squared (R²): 0.008310433702036213


In [186]:
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
mean_cv_score = -cv_scores.mean()
std_cv_score = cv_scores.std()

print("Mean CV Score (MAE):", mean_cv_score)
print("Standard Deviation of CV Scores:", std_cv_score)



Mean CV Score (MAE): 161.88000210180107
Standard Deviation of CV Scores: 6.022589371575634
